In [1]:
import pandas as pd
from scipy.io import loadmat
import matplotlib.pyplot as plt
import pickle
import h5py
import cv2
import os
import numpy as np
import time
import sklearn.preprocessing
from tqdm import tqdm
from matplotlib.backends.backend_pdf import PdfPages
import pylab as pl

from maze_utils import *
from behav_utils import *
from ms_utils import *

mice_status = pd.read_excel("mice_maze_metadataAng.xlsx", sheet_name=0)
mice_names = mice_status["name"].values
data = pd.read_excel("mice_maze_metadataAng.xlsx", sheet_name=2)
if not os.path.exists('data'):
    os.makedirs('data')
if not os.path.exists('img'):
    os.makedirs('img')

In [2]:
# Loop over mice_name, maze_type and training_day
for mice_name in mice_names:
    # maze_type_set = set(data[data.name == mice_name]["maze_type"])
    maze_type_set = {2}
    
    for maze_type in maze_type_set:
        training_day_set = set(data[(data.name == mice_name) & (data.maze_type == maze_type)]["training_day"])

        for i_day in training_day_set:
            current_day = i_day
            current_sign = "M" + str(mice_name)+"maze"+str(maze_type)+"Day"+str(current_day)
            print(current_sign)
            
            folder =  data[(data.name == mice_name) & (data.training_day == current_day) & (data.maze_type == maze_type)]["recording_folder"]
            print(folder)
            
            matfile = folder.values[0] + "/" + "ms.mat"
            # read calcium data: ms.mat
            with h5py.File(matfile, 'r') as f:
                ms_mat = f['ms']
                FiltTraces = np.array(ms_mat['FiltTraces'])
                RawTraces = np.array(ms_mat['RawTraces'])
                DeconvSignal = np.array(ms_mat['DeconvSignals'])
                ms_time = np.array(ms_mat['time'])[0,]
            # read behav data
            decision_file = 'data/' + current_sign +'behav_decision.pkl'
            with open(decision_file, 'rb') as handle:
                correct_time, wrong_time, correct_time_percentage, decision_rate,time_levels, stop_time_mean, stop_time_median, speed_bin = pickle.load(handle)
            behav_processed_file = 'data/' + current_sign +'behav_processed.pkl'
            with open(behav_processed_file, 'rb') as handle:
                behav_time_original, behav_nodes_interpolated, behav_dir = pickle.load(handle)

            #### Preprocessing data
            # remove no behav recording period
            behav_mask = (ms_time>=behav_time_original[0]) & (ms_time<behav_time_original[-1])
            FiltTraces_behav = FiltTraces[:,behav_mask]
            RawTraces_behav = RawTraces[:,behav_mask]
            DeconvSignal_behav = DeconvSignal[:,behav_mask]
            ms_time_behav = ms_time[behav_mask]
            # binarize neuron.S
            deconv_sd = np.std(DeconvSignal_behav, axis = 1) * 3
            Spikes = np.where(DeconvSignal_behav>np.repeat(deconv_sd[:,np.newaxis], DeconvSignal_behav.shape[1],1), 1, 0)

            #### Incorporate maze structure to behavior trajectory: interpolation, etc..
            # transform bin number, making upper right bin (1, 1)
            nx = 12
            ny = 12
            graph = maze1_graph if maze_type==1 else maze2_graph
            total_bin = nx*ny
            start_node = 1
            end_node = 144
            test_maze = Maze(nx, ny, graph)
            new_maze = test_maze.make_maze()
            # new_maze = test_maze.make_maze() # no need to make_maze, already included in __init__()
            shortest_path = test_maze.BFS_SP(graph, start_node, end_node)

            #### Analyze calcium data
            # spike per bin
            spike_nodes = np.zeros_like(Spikes[0,:])
            for i in range(0, len(spike_nodes)):
                if ms_time_behav[i] < behav_time_original[0]:
                    spike_nodes[i] = start_node;
                else:
                    match_index = np.where(behav_time_original <= ms_time_behav[i])[0][-1]
                    spike_nodes[i] = behav_nodes_interpolated[match_index]
            spike_bins = test_maze.idx_to_loc(spike_nodes-1)

            # occupancy map, in ms
            # Duration for each (X,Y) sample (clipped to maxGap)
            maxGap = 100
            stay_time = np.append(np.ediff1d(ms_time_behav),0)
            stay_time[stay_time>maxGap] = maxGap
            _nbins = total_bin
            _coords_range = [0, _nbins +0.0001 ]
            occu_time, xbin_edges, bin_numbers = scipy.stats.binned_statistic(
                    spike_nodes,
                    stay_time,
                    bins=_nbins,
                    statistic="sum",
                    range=_coords_range)

            assert((bin_numbers == spike_nodes).all())
            minimum_occu_thres = 50 # ms, occupancy time less than the threshold are set to NAN, preventing large spike rate
            occu_time[occu_time<minimum_occu_thres] = np.nan
            # rate map (for all neuron)
            _nbins = total_bin
            _coords_range = [0, _nbins +0.0001 ]
            n_neuron = Spikes.shape[0]
            spike_freq_all = np.zeros([n_neuron,total_bin])
            rate_map_all =  np.zeros_like(spike_freq_all)
            for i in range(n_neuron):
                spike_freq_all[i,] ,_ ,_= scipy.stats.binned_statistic(
                    spike_nodes,
                    Spikes[i,:],
                    bins=_nbins,
                    statistic="sum",
                    range=_coords_range)
                rate_map_all[i,] = spike_freq_all[i,]/(occu_time/1000+ 1E-9)

            #### Calculate spatial information (all neurons) and shuffle (shift method)
            t_total = np.nansum(occu_time)/1000 # total time of trial
            t_nodes_frac = occu_time/1000/ (t_total+ 1E-6)   # time spent in ith bin/total session time
            SI_all = np.zeros(n_neuron)
            is_placecell_shift = np.zeros(n_neuron)
            for i in tqdm(range(n_neuron)):
                SI_all[i] = calc_SI(spikes=Spikes[i,], rate_map=rate_map_all[i,], t_total=t_total, t_nodes_frac=t_nodes_frac)
                is_placecell_shift[i] = shuffle_test_shift(SI = SI_all[i], spikes = Spikes[i,], spike_nodes=spike_nodes, occu_time=occu_time, _coords_range=_coords_range)    
            print("# of place cell using time shift shuffle method:" ,np.sum(is_placecell_shift ),"/",n_neuron)

            ## !! save data
            mylist = [rate_map_all, is_placecell_shift, SI_all, spike_freq_all, occu_time]
            with open('data/' + current_sign + "ratemap.pkl", 'wb') as f:
                pickle.dump(mylist, f)


            #### run selective %
            ## spike node to runs
            spike_runs = np.zeros_like(spike_nodes)
            for i in range(len(spike_nodes)):
                spike_runs[i] = node_to_run(spike_nodes[i], test_maze.runs)

            ## occupancy map, in ms
            _nruns = len(test_maze.runs)
            _coords_range = [0, _nruns +0.0001 ]
            occu_time_run, _, run_numbers = scipy.stats.binned_statistic(
                    spike_runs,
                    stay_time,
                    bins=_nruns,
                    statistic="sum",
                    range=_coords_range)
            assert((run_numbers == spike_runs).all())
            minimum_occu_thres = 50 # ms, occupancy time less than the threshold are set to NAN, preventing large spike rate
            occu_time_run[occu_time_run<minimum_occu_thres] = np.nan

            ## run rate map 
            ## sum rate map (for first neuron)
            spikes = Spikes[0,:]
            # spike_map: total spikes per nodes
            spike_freq_run, xbin_edges, run_numbers = scipy.stats.binned_statistic(
                    spike_runs,
                    spikes,
                    bins=_nruns,
                    statistic="sum",
                    range=_coords_range)
            rate_map_run = spike_freq_run/(occu_time_run/1000+ 1E-6) # ratemap: spike_map / occupancy_map
            assert((run_numbers == spike_runs).all())

            ## rate map for runs (for all neuron)
            n_neuron = Spikes.shape[0]
            spike_freq_run_all = np.zeros([n_neuron, _nruns])
            rate_map_run_all =  np.zeros_like(spike_freq_run_all)
            for i in range(n_neuron):
                spike_freq_run_all[i,] ,_ ,_= scipy.stats.binned_statistic(
                    spike_runs,
                    Spikes[i,:],
                    bins=_nruns,
                    statistic="sum",
                    range=_coords_range)
                rate_map_run_all[i,] = spike_freq_run_all[i,]/(occu_time_run/1000+ 1E-9)

            ## spatial information for runs(all neurons)
            t_runs_frac = occu_time_run/1000/ (t_total+ 1E-6)   # time spent in ith bin/total session time

            # Shuffle: shift with a random constant
            SI_run_all = np.zeros(n_neuron)
            is_runcell_shift = np.zeros(n_neuron)

            for i in tqdm(range(n_neuron)):
                SI_run_all[i] = calc_SI(spikes=Spikes[i,], rate_map=rate_map_run_all[i,], t_total=t_total, t_nodes_frac=t_runs_frac)
                is_runcell_shift[i] = shuffle_test_shift(SI = SI_run_all[i], spikes = Spikes[i,], spike_nodes=spike_runs, occu_time=occu_time_run, _coords_range=_coords_range, _nbins = _nruns)

            print("# of run cell using time shift shuffle method:", np.sum(is_runcell_shift), "/", n_neuron)

            ## !! save data
            mylist = [rate_map_run_all, is_runcell_shift, SI_run_all, spike_freq_run_all, occu_time_run]
            with open('data/' + current_sign + "ratemap_run.pkl", 'wb') as f:
                pickle.dump(mylist, f)


M10019maze1Day1
0    Z:/miniscope_recording/maze/4_18_2020/10019H16...
Name: recording_folder_local, dtype: object
The maze is bidirectional, Ye!
Node 1 is Decision point
Node 2 is Leaf node
Node 13 is Decision point
Node 6 is Decision point
Node 8 is Leaf node
Node 17 is Decision point
Node 28 is Leaf node
Node 30 is Decision point
Node 24 is Decision point
Node 84 is Decision point
Node 94 is Decision point
Node 55 is Decision point
Node 43 is Leaf node
Node 54 is Decision point
Node 53 is Leaf node
Node 51 is Decision point
Node 39 is Decision point
Node 40 is Leaf node
Node 97 is Decision point
Node 86 is Leaf node
Node 109 is Decision point
Node 129 is Decision point
Node 139 is Leaf node
Node 142 is Decision point
Node 130 is Decision point
Node 120 is Leaf node
Node 144 is Leaf node
Node 143 is Leaf node
Node 135 is Leaf node
Node 52 is Leaf node
Node 93 is Leaf node
Node 96 is Leaf node
Node 35 is Leaf node
Node 41 is Leaf node
Node 25 is Leaf node
Shortest path =  1 13 14 26 2

100%|██████████████████████████████████████████████████████████████████████████████████| 82/82 [02:29<00:00,  1.82s/it]


# of place cell using time shift shuffle method: 28.0 / 82


100%|██████████████████████████████████████████████████████████████████████████████████| 82/82 [02:23<00:00,  1.74s/it]


# of run cell using time shift shuffle method: 21.0 / 82
M10019maze1Day2
1    Z:/miniscope_recording/maze/4_19_2020/10019H16...
Name: recording_folder_local, dtype: object
The maze is bidirectional, Ye!
Node 1 is Decision point
Node 2 is Leaf node
Node 13 is Decision point
Node 6 is Decision point
Node 8 is Leaf node
Node 17 is Decision point
Node 28 is Leaf node
Node 30 is Decision point
Node 24 is Decision point
Node 84 is Decision point
Node 94 is Decision point
Node 55 is Decision point
Node 43 is Leaf node
Node 54 is Decision point
Node 53 is Leaf node
Node 51 is Decision point
Node 39 is Decision point
Node 40 is Leaf node
Node 97 is Decision point
Node 86 is Leaf node
Node 109 is Decision point
Node 129 is Decision point
Node 139 is Leaf node
Node 142 is Decision point
Node 130 is Decision point
Node 120 is Leaf node
Node 144 is Leaf node
Node 143 is Leaf node
Node 135 is Leaf node
Node 52 is Leaf node
Node 93 is Leaf node
Node 96 is Leaf node
Node 35 is Leaf node
Node 41 is Lea

...

# of run cell using time shift shuffle method: 1.0 / 29
M10033maze1Day8
151    Z:/miniscope_recording/maze/4_25_2020/10033H15...
Name: recording_folder_local, dtype: object
The maze is bidirectional, Ye!
Node 1 is Decision point
Node 2 is Leaf node
Node 13 is Decision point
Node 6 is Decision point
Node 8 is Leaf node
Node 17 is Decision point
Node 28 is Leaf node
Node 30 is Decision point
Node 24 is Decision point
Node 84 is Decision point
Node 94 is Decision point
Node 55 is Decision point
Node 43 is Leaf node
Node 54 is Decision point
Node 53 is Leaf node
Node 51 is Decision point
Node 39 is Decision point
Node 40 is Leaf node
Node 97 is Decision point
Node 86 is Leaf node
Node 109 is Decision point
Node 129 is Decision point
Node 139 is Leaf node
Node 142 is Decision point
Node 130 is Decision point
Node 120 is Leaf node
Node 144 is Leaf node
Node 143 is Leaf node
Node 135 is Leaf node
Node 52 is Leaf node
Node 93 is Leaf node
Node 96 is Leaf node
Node 35 is Leaf node
Node 41 is Le

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:11<00:00,  1.79it/s]


# of place cell using time shift shuffle method: 0.0 / 20


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:11<00:00,  1.69it/s]

# of run cell using time shift shuffle method: 1.0 / 20
